Sumber dataset : https://www.kaggle.com/datasets/himelsarder/cinema-hall-ticket-sales-and-customer-behavior

## Library

In [2]:
import pandas as pd

## Membaca Data

In [3]:
df = pd.read_csv("/content/drive/MyDrive/PANDAS/cinema_hall_ticket_sales.csv")

df

,Ticket_ID,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again
0,N4369,55,12.27,Comedy,Standard,7,No
1,B8091,35,19.02,Drama,Standard,Alone,Yes
2,V6341,55,22.52,Horror,VIP,3,No
3,B3243,53,23.01,Drama,Standard,6,Yes
4,I3814,30,21.81,Comedy,VIP,4,Yes
...,...,...,...,...,...,...,...
1435,E5406,41,17.09,Sci-Fi,Standard,2,Yes
1436,C4357,19,11.45,Action,Standard,Alone,No
1437,R7285,53,17.13,Comedy,Premium,Alone,Yes
1438,W6360,34,20.25,Comedy,Premium,2,Yes


# **DATA UNDERSTUNDING**

## Pengecekan Struktur Data

In [22]:
print("\nTipe data setiap kolom dalam data ini: ")
df.dtypes


Tipe data setiap kolom dalam data ini: 


,0
Age,int64
Ticket_Price,float64
Movie_Genre,object
Seat_Type,object
Number_of_Person,int64
Purchase_Again,int64
Total_Ticket_Price,float64


In [5]:
print("\nJumlah baris dan kolom dalam data ini: ")
df.shape


Jumlah baris dan kolom dalam data ini: 


(1440, 7)

In [23]:
print("\nStatistik deskriptif data ini: ")
df.describe()


Statistik deskriptif data ini: 


,Age,Ticket_Price,Number_of_Person,Purchase_Again,Total_Ticket_Price
count,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000
mean,38.904861,17.383972,2.881944,0.490972,50.050250
std,12.426293,4.397087,2.192591,0.500092,41.465089
min,18.000000,10.010000,1.000000,0.000000,10.020000
25%,28.000000,13.640000,1.000000,0.000000,17.760000
50%,39.000000,17.280000,2.000000,0.000000,26.350000
75%,50.000000,21.392500,5.000000,1.000000,74.975000
max,60.000000,24.990000,7.000000,1.000000,174.720000


In [24]:
print('\nKorelasi antar kolom numerik: ')
df.corr(numeric_only=True)


Korelasi antar kolom numerik: 


,Age,Ticket_Price,Number_of_Person,Purchase_Again,Total_Ticket_Price
Age,1.000000,0.036528,-0.010283,0.023401,0.006286
Ticket_Price,0.036528,1.000000,-0.005127,-0.022706,0.304393
Number_of_Person,-0.010283,-0.005127,1.000000,-0.015549,0.921225
Purchase_Again,0.023401,-0.022706,-0.015549,1.000000,-0.022338
Total_Ticket_Price,0.006286,0.304393,0.921225,-0.022338,1.000000


# **DATA PREPARATION**

## Missing Values

In [25]:
print((df.isna().sum() / len(df)) * 100)

Age                   0.0
Ticket_Price          0.0
Movie_Genre           0.0
Seat_Type             0.0
Number_of_Person      0.0
Purchase_Again        0.0
Total_Ticket_Price    0.0
dtype: float64


# Duplicated Values

In [9]:
df[df.duplicated()]

,Ticket_ID,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again


# Outliers

In [26]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
Age,0.000000
Ticket_Price,0.000000
Number_of_Person,0.000000
Purchase_Again,0.000000
Total_Ticket_Price,1.666667


In [27]:
df

,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again,Total_Ticket_Price
0,55,12.27,Comedy,Standard,7,0,85.89
1,35,19.02,Drama,Standard,1,1,19.02
2,55,22.52,Horror,VIP,3,0,67.56
3,53,23.01,Drama,Standard,6,1,138.06
4,30,21.81,Comedy,VIP,4,1,87.24
...,...,...,...,...,...,...,...
1435,41,17.09,Sci-Fi,Standard,2,1,34.18
1436,19,11.45,Action,Standard,1,0,11.45
1437,53,17.13,Comedy,Premium,1,1,17.13
1438,34,20.25,Comedy,Premium,2,1,40.50


Melakukan Imputasi

In [29]:
columns_to_impute = ["Total_Ticket_Price"]

for col in columns_to_impute:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Menggunakan .loc[] agar tidak muncul SettingWithCopyWarning
    df.loc[:, col] = df[col].clip(lower=lower_bound, upper=upper_bound)

In [30]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
Age,0.0
Ticket_Price,0.0
Number_of_Person,0.0
Purchase_Again,0.0
Total_Ticket_Price,0.0


In [31]:
df

,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again,Total_Ticket_Price
0,55,12.27,Comedy,Standard,7,0,85.89
1,35,19.02,Drama,Standard,1,1,19.02
2,55,22.52,Horror,VIP,3,0,67.56
3,53,23.01,Drama,Standard,6,1,138.06
4,30,21.81,Comedy,VIP,4,1,87.24
...,...,...,...,...,...,...,...
1435,41,17.09,Sci-Fi,Standard,2,1,34.18
1436,19,11.45,Action,Standard,1,0,11.45
1437,53,17.13,Comedy,Premium,1,1,17.13
1438,34,20.25,Comedy,Premium,2,1,40.50


## Incosistent Value

In [15]:
df

,Ticket_ID,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again
0,N4369,55,12.27,Comedy,Standard,7,No
1,B8091,35,19.02,Drama,Standard,Alone,Yes
2,V6341,55,22.52,Horror,VIP,3,No
3,B3243,53,23.01,Drama,Standard,6,Yes
4,I3814,30,21.81,Comedy,VIP,4,Yes
...,...,...,...,...,...,...,...
1435,E5406,41,17.09,Sci-Fi,Standard,2,Yes
1436,C4357,19,11.45,Action,Standard,Alone,No
1437,R7285,53,17.13,Comedy,Premium,Alone,Yes
1438,W6360,34,20.25,Comedy,Premium,2,Yes


Mengubah pada kolom Number_of_Person yang sebelumnya terdapat objek "Alone" akan diubah menjadi 1.

In [16]:
df["Number_of_Person"] = pd.to_numeric(df["Number_of_Person"], errors="coerce")
df["Number_of_Person"] = df["Number_of_Person"].fillna(0).astype(int)
df["Number_of_Person"] = df['Number_of_Person'].replace(0,1)

df

,Ticket_ID,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again
0,N4369,55,12.27,Comedy,Standard,7,No
1,B8091,35,19.02,Drama,Standard,1,Yes
2,V6341,55,22.52,Horror,VIP,3,No
3,B3243,53,23.01,Drama,Standard,6,Yes
4,I3814,30,21.81,Comedy,VIP,4,Yes
...,...,...,...,...,...,...,...
1435,E5406,41,17.09,Sci-Fi,Standard,2,Yes
1436,C4357,19,11.45,Action,Standard,1,No
1437,R7285,53,17.13,Comedy,Premium,1,Yes
1438,W6360,34,20.25,Comedy,Premium,2,Yes


Mengubah tipe data yang sebelumnya string menjadi int pada kolom Purchase_Again.

In [19]:
df['Purchase_Again'] = df['Purchase_Again'].replace({'Yes': 1, 'No': 0}).astype(int)

df

,Ticket_ID,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again
0,N4369,55,12.27,Comedy,Standard,7,0
1,B8091,35,19.02,Drama,Standard,1,1
2,V6341,55,22.52,Horror,VIP,3,0
3,B3243,53,23.01,Drama,Standard,6,1
4,I3814,30,21.81,Comedy,VIP,4,1
...,...,...,...,...,...,...,...
1435,E5406,41,17.09,Sci-Fi,Standard,2,1
1436,C4357,19,11.45,Action,Standard,1,0
1437,R7285,53,17.13,Comedy,Premium,1,1
1438,W6360,34,20.25,Comedy,Premium,2,1


## Construct Data

In [20]:
df['Total_Ticket_Price'] = df['Ticket_Price'] * df['Number_of_Person']

print(df.head())

  Ticket_ID  Age  Ticket_Price Movie_Genre Seat_Type  Number_of_Person  \
0     N4369   55         12.27      Comedy  Standard                 7   
1     B8091   35         19.02       Drama  Standard                 1   
2     V6341   55         22.52      Horror       VIP                 3   
3     B3243   53         23.01       Drama  Standard                 6   
4     I3814   30         21.81      Comedy       VIP                 4   

   Purchase_Again  Total_Ticket_Price  
0               0               85.89  
1               1               19.02  
2               0               67.56  
3               1              138.06  
4               1               87.24  


## Data Reduction

In [21]:
df = df.drop('Ticket_ID', axis=1, errors='ignore')

df

,Age,Ticket_Price,Movie_Genre,Seat_Type,Number_of_Person,Purchase_Again,Total_Ticket_Price
0,55,12.27,Comedy,Standard,7,0,85.89
1,35,19.02,Drama,Standard,1,1,19.02
2,55,22.52,Horror,VIP,3,0,67.56
3,53,23.01,Drama,Standard,6,1,138.06
4,30,21.81,Comedy,VIP,4,1,87.24
...,...,...,...,...,...,...,...
1435,41,17.09,Sci-Fi,Standard,2,1,34.18
1436,19,11.45,Action,Standard,1,0,11.45
1437,53,17.13,Comedy,Premium,1,1,17.13
1438,34,20.25,Comedy,Premium,2,1,40.50


## Menyimpan Data

In [32]:
output_path = "/content/drive/MyDrive/PANDAS/cinema_hall_ticket_sales.xlsx"
df.to_excel(output_path, index=False)

print("\nData berhasil disimpan sebagai file Excel:", output_path)


Data berhasil disimpan sebagai file Excel: /content/drive/MyDrive/PANDAS/cinema_hall_ticket_sales.xlsx
